In [ ]:
import os
from bs4 import BeautifulSoup

In [ ]:
doc_dirs = []
for n in range(1,6):
    dirpath = "/srv/data/greek/OGA/oga/oga_v0.2.0_{}/".format(str(n))
    docdir_names = os.listdir(dirpath.format(str(n)))
    doc_dirs.extend([dirpath + docdir_name for docdir_name in docdir_names])
len(doc_dirs)

In [ ]:

doc_dirs[1000:1050]

In [ ]:
doc_dir = "tlg0014.tlg005.perseus-grc2"

In [ ]:
os.listdir(doc_dirs[0])

In [ ]:

file_path = doc_dirs[0] + "/tlg0007.tlg082.perseus-grc3.tok01_cts01.xml"
# Open the XML file
with open(file_path, 'r', encoding='utf-8') as file:
    # Parse the file using BeautifulSoup
    soup_fealist = BeautifulSoup(file, 'xml')  # Use 'xml' parser for XML data

# Print the root tag to confirm loading
print(soup_fealist.prettify())

In [ ]:

file_path = doc_dirs[0] + "/tlg0007.tlg082.perseus-grc3.tok01.xml"
# Open the XML file
with open(file_path, 'r', encoding='utf-8') as file:
    # Parse the file using BeautifulSoup
    soup_tokens = BeautifulSoup(file, 'xml')  # Use 'xml' parser for XML data

# Print the root tag to confirm loading
print(soup_tokens.prettify()[:10000])

In [ ]:
file_path = doc_dirs[0] + "/tlg0007.tlg082.perseus-grc3.tok01_lemma01.xml"
# Open the XML file
with open(file_path, 'r', encoding='utf-8') as file:
    # Parse the file using BeautifulSoup
    soup_tokens = BeautifulSoup(file, 'xml')  # Use 'xml' parser for XML data

# Print the root tag to confirm loading
print(soup_tokens.prettify()[:10000])

In [ ]:
file_path = doc_dirs[0] + "/postag.tlg0007.tlg082.perseus-grc3.tok01_postag01.xml"
# Open the XML file
with open(file_path, 'r', encoding='utf-8') as file:
    # Parse the file using BeautifulSoup
    soup_postags = BeautifulSoup(file, 'xml')  # Use 'xml' parser for XML data

# Print the root tag to confirm loading
print(soup_postags.prettify()[:10000])

In [77]:
from bs4 import BeautifulSoup, Comment

# Define file paths
files = {
    "tokens": doc_dirs[0] + "/tlg0007.tlg082.perseus-grc3.tok01.xml",
    "lemmas": doc_dirs[0] + "/tlg0007.tlg082.perseus-grc3.tok01_lemma01.xml",
    "pos": doc_dirs[0] + "/postag.tlg0007.tlg082.perseus-grc3.tok01_postag01.xml",
    "cts": doc_dirs[0] + "/tlg0007.tlg082.perseus-grc3.tok01_cts01.xml"
}


# Utility to extract positional indices from xlink:href in tokens
def extract_position(xlink):
    pos = xlink.split("string-range(//body,'',")[-1].strip("))").split(",")
    return tuple(map(int, pos))  # Converts "1,9" to (1, 9)


# Parse all files in parallel
with open(files["tokens"], "r", encoding="utf-8") as f_tokens, \
        open(files["lemmas"], "r", encoding="utf-8") as f_lemmas, \
        open(files["pos"], "r", encoding="utf-8") as f_pos, \
        open(files["cts"], "r", encoding="utf-8") as f_cts:
    # Parse XML files
    soup_tokens = BeautifulSoup(f_tokens, "xml")
    soup_lemmas = BeautifulSoup(f_lemmas, "xml")
    soup_pos = BeautifulSoup(f_pos, "xml")
    soup_cts = BeautifulSoup(f_cts, "xml")

    # Extract data
    # Tokens
    tokens_data = {}
    for mark in soup_tokens.find_all("mark"):
        token_id = mark["id"]

        # Extract token from adjacent XML comment
        token_comment = mark.find_next(string=lambda text: isinstance(text, Comment))
        token_string = token_comment.strip() if token_comment else ""  # Handle missing comments gracefully

        tokens_data[token_id] = {
            "token": token_string,
            "pos_index": extract_position(mark["xlink:href"])
        }

    # Lemmas
    for feat in soup_lemmas.find_all("feat"):
        token_id = feat["xlink:href"].lstrip("#")
        if token_id in tokens_data:
            tokens_data[token_id]["lemma"] = feat["value"]

    # POS tags
    for multiFeat in soup_pos.find_all("multiFeat"):
        token_id = multiFeat["xlink:href"].lstrip("#")
        if token_id in tokens_data:
            tokens_data[token_id]["pos"] = multiFeat.find("feat", {"name": "pos"})["value"]

    # Subunits/Sentence IDs
    for feat in soup_cts.find_all("feat"):
        token_id = feat["xlink:href"].lstrip("#")
        if token_id in tokens_data:
            tokens_data[token_id]["subunit"] = feat["value"]

# Combine into list of tuples
token_tuples = [
    (
        data.get("token", ""),
        data.get("lemma", ""),
        data.get("pos", ""),
        data.get("subunit", ""),
        data.get("pos_index", ())
    )
    for data in tokens_data.values()
]

# Output as requested
print(token_tuples)

[('Ἀγασικλῆς', 'ἀγασικλῆς', 'n', '1_1', (1, 9)), ('ὁ', 'ὁ', 'l', '1_1', (11, 1)), ('Λακεδαιμονίων', 'λακεδαιμόνιος', 'n', '1_1', (13, 13)), ('βασιλεύς', 'βασιλεύς', 'n', '1_1', (27, 8)), (',', ',', 'u', '1_1', (36, 1)), ('θαυμάζοντός', 'θαυμάζω', 'v', '1_1', (38, 11)), ('τινος', 'τις', 'p', '1_1', (50, 5)), ('ὅτι', 'ὅτι', 'c', '1_1', (56, 3)), ('φιλήκοος', 'φιλήκοος', 'a', '1_1', (60, 8)), ('ὢν', 'εἰμί', 'v', '1_1', (69, 2)), ('οὐ', 'οὐ', 'd', '1_1', (72, 2)), ('προσδέχεται', 'προσδέχομαι', 'v', '1_1', (75, 11)), ('Φιλοφάνη', 'φιλοφάνης', 'n', '1_1', (87, 8)), ('τὸν', 'ὁ', 'l', '1_1', (96, 3)), ('σοφιστήν', 'σοφιστής', 'n', '1_1', (100, 8)), (',', ',', 'u', '1_1', (109, 1)), ('ἔφη', 'φημί', 'v', '1_1', (111, 3)), (',', ',', 'u', '1_1', (115, 1)), ('τούτων', 'οὗτος', 'p', '1_1', (117, 6)), ('χρῄζω', 'χρῄζω', 'v', '1_1', (124, 5)), ('μαθητὴς', 'μαθητής', 'n', '1_1', (130, 7)), ('εἶναι', 'εἰμί', 'v', '1_1', (138, 5)), (',', ',', 'u', '1_1', (144, 1)), ('ὧν', 'ὅς', 'p', '1_1', (146, 2)), (